In [ ]:
from __future__ import print_function
import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings('ignore')
from include import *
from PIL import Image
import PIL
import pywt
import numpy as np
import torch
import torchvision
from torch.autograd import Variable
import time
import copy





GPU = True
if GPU == True:
    torch.backends.cudnn.enabled = True
    torch.backends.cudnn.benchmark = True
    dtype = torch.cuda.FloatTensor
    os.environ['CUDA_VISIBLE_DEVICES'] = '0'
    print("num GPUs",torch.cuda.device_count())
    device = 'cuda'
    if torch.cuda.device_count()==0:
        dtype = torch.FloatTensor
        device = 'cpu'
else:
    dtype = torch.FloatTensor
    device = 'cpu'



# 1. Load a test image from a dataset (now : CelebA 128x128)

In [ ]:
#dataset = 'mnist' # 'mnist' or 'celeba'
dataset = 'celeba'
path = './test_data/' + dataset + '/' 
img_name = dataset + '2' # 1-5 (for celeba), 1-6 (for mnist)
img_path = path + img_name + ".jpg"
img_pil = Image.open(img_path)

if dataset == 'celeba':
    cx=89
    cy=121
    img_pil = img_pil.crop((cx-64, cy - 64, cx + 64, cy+64))

img_np = pil_to_np(img_pil)
print('Dimensions of input image:', img_np.shape)
img_np = img_np / np.max(img_np)


img_np_orig = 1*img_np

'''if dataset == 'celeba':
    plt.imshow(img_np.transpose(1,2,0))
else:
    plt.imshow(img_np[0,:,:])
    plt.gray()'''

img_var = np_to_var(img_np).type(dtype)
d = img_np.shape[1]
out_ch = img_np.shape[0]
d_image = img_np.size

# normalize the pixels to [-1,1]
img_var = 2*img_var -1

grid = torchvision.utils.make_grid(img_var.clamp(min=-1, max=1), scale_each=True, normalize=True)
plt.imshow(grid.detach().permute(1, 2, 0).cpu().numpy())    
plt.axis('off')

save_to = './recovery/'
save_path= save_to + 'Original'+'_'+img_name+'.png'
plt.savefig(save_path, bbox_inches='tight', pad_inches = 0)



In [ ]:
f = 0.3 #compression rate
print('Compression rate is ', f)
m_image = int(f*d_image)
print('Number of measurements is ',m_image, ' for signal of length ', d_image)

# random Gaussian measurement process

A = torch.randn(m_image, d_image).to(device)/np.sqrt(m_image)
x = img_var.to(device).reshape(d_image)
y = torch.matmul(A,x).to(device)

#noise = 0.1 * torch.randn(m_image).to(device) 
#y = y+ noise

#latentDim = model.config.noiseVectorDim
print(A.shape, x.shape, y.shape)
mse = torch.nn.MSELoss()

# 2. Compressed sensing using generative models

## 2.1. Load a pretrained generative model on the dataset (now: PGGAN)

In [ ]:
use_gpu = True if torch.cuda.is_available() else False

# trained on high-quality celebrity faces "celebA" dataset
# this model outputs 512 x 512 pixel images
model = torch.hub.load('facebookresearch/pytorch_GAN_zoo:hub',
                       'PGAN', model_name='celeba',
                       pretrained=True, useGPU=use_gpu)
# this model outputs 256 x 256 pixel images
# model = torch.hub.load('facebookresearch/pytorch_GAN_zoo:hub',
#                        'PGAN', model_name='celebAHQ-256',
#                        pretrained=True, useGPU=use_gpu)
G = model.netG

#G.eval()
latentDim = model.config.noiseVectorDim

## 2.2. CS using the loaded GAN

In [ ]:
t0= time.time()

z0, mse_wrt_loss = CSGM(G, latentDim, y, A, device, num_iter = 1600, lr=0.1, lr_decay_epoch=0, factor = 0.7)
x0 = G(z0)

grid = torchvision.utils.make_grid(x0.clamp(min=-1, max=1), scale_each=True, normalize=True)
plt.axis('off')
plt.imshow(grid.detach().permute(1, 2, 0).cpu().numpy())

save_path= save_to +'PGGAN'+'_'+img_name+'.png'
plt.savefig(save_path,bbox_inches='tight', pad_inches = 0) 
    
t1= time.time()
print('\nTime elapsed:',t1-t0)

error_wrt_truth = mse(x0, img_var).item()
print('\nl2-recovery error:', error_wrt_truth)

In [ ]:
plt.xlabel('optimizer iteration')
plt.ylabel('recovery error')
plt.semilogy(mse_wrt_loss)

# 3. Compressed Sensing using Deep decoder

## 3.1. Define the network

In [ ]:
#use decoder architecture or DC GAN architecture
decodetype = 'upsample' # transposeconv / upsample

num_channels = [10,20,40,60,80] 

output_depth = img_np.shape[0] # number of output channels
net = autoencodernet(num_output_channels=output_depth,num_channels_up=num_channels,need_sigmoid=True, 
                        decodetype=decodetype
                        ).type(dtype)

print("number of parameters: ", num_param(net))
if decodetype == 'upsample':
    print(net.decoder)
elif decodetype == 'transposeconv':
    print(net.convdecoder)
net_in = copy.deepcopy(net)



## 3.2. CS using untrained network

In [ ]:
t0 = time.time()

net, net_input, loss = CS_DD(net, num_channels, d_image, y=y, A=A, device= device, 
                             num_iter =8000, lr = 0.0002, lr_decay_epoch=3000, factor = 0.7, decodetype = decodetype)
x_DD = 2*net( net_input.type(dtype) )-1 #.data.cpu().numpy()[0]

t1 = time.time()
grid = torchvision.utils.make_grid(x_DD.clamp(min=-1, max=1), scale_each=True, normalize=True)
plt.imshow(grid.detach().permute(1, 2, 0).cpu().numpy())
plt.axis('off')
#plt.imshow(x_hat.transpose(1,2,0))
#plt.show()

print('\n time elapsed:', t1-t0)

error_wrt_truth = mse(x_DD, img_var).item()
print('\nl2-recovery error:', error_wrt_truth)

save_path= save_to +'DD'+'_'+img_name+'.png'
plt.savefig(save_path,bbox_inches='tight', pad_inches = 0)


In [ ]:
plt.xlabel('optimizer iteration')
plt.ylabel('recovery error')
plt.semilogy(loss)

# 4. Compressed sensing using hybrid model

## 4.1. Define the untrained network used for hybrid model

In [ ]:
#use decoder architecture or DC GAN architecture
decodetype = 'upsample' # transposeconv / upsample

num_channels = [40,40,40,40,40,40] 

output_depth = img_np.shape[0] # number of output channels
net = autoencodernet(num_output_channels=output_depth,num_channels_up=num_channels,need_sigmoid=True, 
                        decodetype=decodetype
                        ).type(dtype)

print("number of parameters: ", num_param(net))
#if decodetype == 'upsample':
#    print(net.decoder)
#elif decodetype == 'transposeconv':
#    print(net.convdecoder)
#net_copy = copy.deepcopy(net)

## 4.2. CS using hybrid model

In [ ]:
t0 = time.time()

#initialization by csgm
print('Initialization step')
z0, loss1 = CSGM(G=G, latentDim=latentDim, y=y, A=A, device=device, num_iter=500, lr = 0.1, lr_decay_epoch = 0)
x0 = G(z0)
print('error = ', mse(x0, img_var))

net, net_input, loss2 = CS_DD(net, num_channels, d_image, y=y, A=A, device= device, 
                             num_iter = 500, lr = 0.0002, lr_decay_epoch=0, decodetype = decodetype)
x_DD = 2*net( net_input.type(dtype) )-1 #.data.cpu().numpy()[0]
print('error = ',  mse(x_DD, img_var).item())

# performing optimization
print('Performing optimization')
net, net_input, z, alpha, beta, loss3 = CS_hybrid2(G, net, net_input, num_channels, d_image, y, A, device, 
              z0, latentDim, num_iter = 6000, lr_decay_epoch = 2000, factor = 0.7,
              decodetype = decodetype, lr_alpha = 0.01, lr_beta = 0.01, lr_z = 0.05, lr_net = 0.0001, 
              alpha_init = 0.5, beta_init = 0.5)
x_hat = alpha.clamp(0,1)*G(z) + beta.clamp(0,1)*(2*net(net_input.type(dtype)) - 1)
print('error = ', mse(x_hat, img_var).item())

print(alpha,beta)

grid = torchvision.utils.make_grid(x_hat.clamp(min=-1, max=1), scale_each=True, normalize=True)
plt.imshow(grid.detach().permute(1, 2, 0).cpu().numpy())
plt.axis('off')

t1 = time.time()
print('\n time elapsed:', t1-t0)

error_wrt_truth = mse(x_hat, img_var).item()
print('\nl2-recovery error:', error_wrt_truth)


save_path= save_to +'Hybrid'+'_'+img_name+'.png'
plt.savefig(save_path,bbox_inches='tight', pad_inches = 0) 

In [ ]:
plt.xlabel('optimizer iteration')
plt.ylabel('recovery error')
loss = np.concatenate((loss1,loss2,loss3), axis = 0)
plt.semilogy(loss3)

##4.3. Using default Adam optimizer

In [ ]:
t0 = time.time()

#initialization by csgm
print('Initialization step')
z0, loss1 = CSGM(G=G, latentDim=latentDim, y=y, A=A, device=device, num_iter=500, lr = 0.2, lr_decay_epoch = 0)
x0 = G(z0)
print('error = ', mse(x0, img_var))

net, net_input, loss2 = CS_DD(net, num_channels, d_image, y=y, A=A, device= device, 
                             num_iter = 500, lr = 0.0002, lr_decay_epoch=0)
x_DD = 2*net( net_input.type(dtype) )-1 #.data.cpu().numpy()[0]
print('error = ',  mse(x_DD, img_var).item())

# performing optimization
print('Performing optimization')
net, net_input, z, alpha, beta, loss3 = CS_hybrid_Adam(G, net, net_input, num_channels, d_image, y, A, device, 
              z0, latentDim, num_iter = 5000, decodetype = 'upsample')
x_hat = alpha.clamp(0,1)*G(z) + beta.clamp(0,1)*(2*net(net_input.type(dtype)) - 1)
print('error = ', mse(x_hat, img_var).item())

print(alpha,beta)

grid = torchvision.utils.make_grid(x_hat.clamp(min=-1, max=1), scale_each=True, normalize=True)
plt.imshow(grid.detach().permute(1, 2, 0).cpu().numpy())
plt.axis('off')

t1 = time.time()
print('\n time elapsed:', t1-t0)

error_wrt_truth = mse(x_hat, img_var).item()
print('\nl2-recovery error:', error_wrt_truth)


save_path= save_to +'Hybrid'+'_'+img_name+'.png'
plt.savefig(save_path,bbox_inches='tight', pad_inches = 0) 

In [ ]:
plt.xlabel('optimizer iteration')
plt.ylabel('recovery error')
loss = np.concatenate((loss1,loss2,loss3), axis = 0)
plt.semilogy(loss3)

#5. Image adaptive GAN

In [ ]:
G_IA = copy.deepcopy(G)

t0= time.time()

z0, mse_wrt_loss = CSGM(G, latentDim, y, A, device, num_iter = 500, lr=0.2, lr_decay_epoch=0, factor = 0.7)
x0 = G(z0)

z_IA, mse_wrt_loss = IA(G_IA, latentDim, y, A, z0, device, num_iter= 5000)
x_IA = G_IA(z_IA)

grid = torchvision.utils.make_grid(x_IA.clamp(min=-1, max=1), scale_each=True, normalize=True)
plt.axis('off')
plt.imshow(grid.detach().permute(1, 2, 0).cpu().numpy())

save_path= save_to +'IAGAN'+'_'+img_name+'.png'
plt.savefig(save_path,bbox_inches='tight', pad_inches = 0) 
    
t1= time.time()
print('\nTime elapsed:',t1-t0)

error_wrt_truth = mse(x_IA, img_var).item()
print('\nl2-recovery error:', error_wrt_truth)